# Part 1 word embedding model

In [ ]:
!pip3 install wget

In [1]:
import wget

In [ ]:
import wget
url = 'https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json'
filename = wget.download(url)

In [1]:
import json  
with open('train-v2.0.json', 'r') as f:    
    train_data = json.load(f)
    

In [2]:
train_data=[item for topic in train_data['data'] for item in topic['paragraphs']]

In [3]:
from pandas.io.json import json_normalize
import pandas as pd

In [5]:
df = pd.DataFrame.from_dict(json_normalize(train_data), orient='columns')

In [6]:
df.head(10)

,context,qas
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,[{'question': 'When did Beyonce start becoming...
1,Following the disbandment of Destiny's Child i...,"[{'question': 'After her second solo album, wh..."
2,"A self-described ""modern-day feminist"", Beyonc...","[{'question': 'In her music, what are some rec..."
3,"Beyoncé Giselle Knowles was born in Houston, T...",[{'question': 'Beyonce's younger sibling also ...
4,Beyoncé attended St. Mary's Elementary School ...,[{'question': 'What town did Beyonce go to sch...
5,"At age eight, Beyoncé and childhood friend Kel...",[{'question': 'Who decided to place Beyonce's ...
6,The group changed their name to Destiny's Chil...,[{'question': 'Which film featured Destiny's C...
7,LeToya Luckett and Roberson became unhappy wit...,[{'question': 'What mental health issue did Be...
8,"The remaining band members recorded ""Independe...","[{'question': '""Charlie's Angels"" featured whi..."
9,"In July 2002, Beyoncé continued her acting car...",[{'question': 'Who did Beyonce star with in th...


In [33]:
ctxt=[]
q1=[]
idd1=[]
anw1=[]
is_impossible1=[]
for i in range(len(df['qas'])):
    r=df.iloc[i]['context']
    qa=df.iloc[i]['qas']
    for j in qa:
        for k,v in j.items():
            #print("La clé {} contient la valeur {}.".format(k, v))
            if k=='question':
                q1.append(v)
                ctxt.append(r)
            if k=='id':
                idd1.append(v)
            if k=='answers':
                anw1.append(v)
            if k=='is_impossible':
                is_impossible1.append(v)
            

In [34]:
anwtxt=[]
anwst=[]
for i in anw1:
    for j in i:
        for key,val in j.items():
            #print("La clé {} contient la valeur {}.".format(key, val))
            if key=='text':
                anwtxt.append(val) 
            if key=='answer_start':
                anwst.append(val)

In [35]:
dfQA = pd.DataFrame(list(zip(ctxt,q1, anwtxt,anwst,is_impossible1 )), columns =['context','question','answer','answerstart','is_impossible'])

In [36]:
dfQA.head(10)

,context,question,answer,answerstart,is_impossible
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s,269,False
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,207,False
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,526,False
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas",166,False
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s,276,False
5,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what R&B group was she the lead singer?,Destiny's Child,320,False
6,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What album made her a worldwide known artist?,Dangerously in Love,505,False
7,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Who managed the Destiny's Child group?,Mathew Knowles,360,False
8,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyoncé rise to fame?,late 1990s,276,False
9,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What role did Beyoncé have in Destiny's Child?,lead singer,290,False


In [11]:
import re
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
def clean(text):
    text = text.lower()
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text

df['process_context'] = df['context'].apply(clean)


In [12]:
# Preparing the dataset

from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
df['process_context_SentdTok']= [sent_tokenize(str(entry)) for entry in df['process_context']]
df['process_context_WordTok']=[word_tokenize(str(entry)) for entry in df['process_context']]


# word embedding function

In [13]:
#Ne marche pas pour une raison inexplicable
from gensim.models import Word2Vec

In [22]:
W2model= Word2Vec(df['process_context_WordTok'], min_count=5,iter=50)
W2model.save("W2v.model")
print("Model Saved")

Model Saved


In [23]:
sim_words = W2model.wv.most_similar("music")
sim_words

[('genre', 0.5759110450744629),
 ('genres', 0.564185619354248),
 ('melodies', 0.544532835483551),
 ('jazz', 0.5368236899375916),
 ('lyrics', 0.5237343311309814),
 ('songs', 0.5207332372665405),
 ('musical', 0.5198172330856323),
 ('pop', 0.5194613933563232),
 ('folk', 0.5117992758750916),
 ('mandolin', 0.5117467641830444)]

In [15]:
from gensim.models import word2vec
tokenized_sentences = [sentence.split() for sentence in df['process_context'] ]
modelSent = word2vec.Word2Vec(tokenized_sentences, min_count=5,iter=50)

In [16]:
sim_words = modelSent.wv.most_similar("music")
sim_words

[('jazz', 0.5492678880691528),
 ('genres', 0.5483729839324951),
 ('songs', 0.5438870191574097),
 ('genre', 0.5422179698944092),
 ('instrumental', 0.5360028743743896),
 ('musical', 0.5308642983436584),
 ('dance', 0.5256602764129639),
 ('lyrics', 0.5146111845970154),
 ('melodies', 0.514244794845581),
 ('musicians', 0.513271689414978)]

In [ ]:
import nltk

In [14]:
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from os import listdir
from os.path import isfile, join
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [ ]:
#same cleaning to be equale 
#so i do not clean i keep the word tokenisation

In [76]:
modelDc = Doc2Vec.load("d2v.model")

In [15]:
 hyperparams  = {
        'size': [10,50,100, 200],
        'min_count': [2,5,15,20],
        'iter': [20,50,100,150],
        'window': [4, 5, 6, 7],
        'alpha': [0.025,0.01,0.03,0.05],
    }

In [91]:
def start_training(hyperparams, train_corpus):
    tagged = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(train_corpus)]
    for i in range (0,4):
        model = Doc2Vec(size=hyperparams['size'][i], min_count=hyperparams['min_count'][i], iter=hyperparams['iter'][i], workers=4, window=hyperparams['window'][i], alpha=hyperparams['alpha'][i])
        print("Building vocabulary")
        print("size : {} min count : {} ite : {} windows {} alpha value {}".format(hyperparams['size'][i],hyperparams['min_count'][i],hyperparams['iter'][i],hyperparams['window'][i],hyperparams['alpha'][i]))
        #model.random.seed(0)
        model.build_vocab(tagged)
        print("Training the model")
        print(model)
        model.train(tagged, total_examples=model.corpus_count, epochs=model.iter)
        model.save("d2v.model"+str(i))
        print("Model Saved")
    return tagged

In [92]:
train_corpus=dfQA['context']
tg=start_training(hyperparams, train_corpus)

//anaconda3/lib/python3.7/site-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
//anaconda3/lib/python3.7/site-packages/gensim/models/doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


Building vocabulary
size : 10 min count : 2 ite : 20 windows 4 alpha value 0.025
Training the model
Doc2Vec(dm/m,d10,n5,w4,mc2,s0.001,t4)


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  # This is added back by InteractiveShellApp.init_path()


Model Saved
Building vocabulary
size : 50 min count : 5 ite : 50 windows 5 alpha value 0.01


//anaconda3/lib/python3.7/site-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
//anaconda3/lib/python3.7/site-packages/gensim/models/doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


...

Model Saved
Building vocabulary
size : 200 min count : 20 ite : 150 windows 7 alpha value 0.05


//anaconda3/lib/python3.7/site-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
//anaconda3/lib/python3.7/site-packages/gensim/models/doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


Training the model
Doc2Vec(dm/m,d200,n5,w7,mc20,s0.001,t4)


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  # This is added back by InteractiveShellApp.init_path()


Model Saved


In [93]:
from gensim.models.doc2vec import Doc2Vec
model0= Doc2Vec.load("d2v.model0")
model1=Doc2Vec.load("d2v.model1")
model2=Doc2Vec.load("d2v.model2")
model3=Doc2Vec.load("d2v.model3")

In [103]:
import wget
url='https://raw.githubusercontent.com/tmikolov/word2vec/master/questions-words.txt'
compTxt = wget.download(url)

#W2v.model

In [104]:
from gensim.test.utils import datapath
similaritiesD20 = model0.wv.evaluate_word_pairs(datapath('wordsim353.tsv'))
similaritiesD21 =model1.wv.evaluate_word_pairs(datapath('wordsim353.tsv'))
similaritiesD22=model2.wv.evaluate_word_pairs(datapath('wordsim353.tsv'))
similaritiesD23=model3.wv.evaluate_word_pairs(datapath('wordsim353.tsv'))

In [105]:
#deux premier chiffre pearson (tuple of (float, float)) – Pearson correlation coefficient with 2-tailed p-value.
#interpret person correlation positive de 0.3 ce qui
#a p-value ( ou niveau de significativité) de la corrélation 
#spearman (tuple of (float, float)) – Spearman rank-order correlation coefficient between the similarities from the dataset and the similarities produced by the model itself, with 2-tailed p-value
#le dernier chiffre The ratio of pairs with unknown words.
print("Model 0 correlation :"+str(similaritiesD20)+"\n"+"\n"+"Model 1 coorelation"+str(similaritiesD21)+"\n"+"\n"+"Model 2 coorelation"+str(similaritiesD22)+"\n"+"\n"+"Model 3 coorelation"+str(similaritiesD23))



Model 0 correlation :((0.27977369647558953, 2.100635275759354e-07), SpearmanrResult(correlation=0.2978394411490589, pvalue=3.0096484808786666e-08), 5.6657223796034)

Model 1 coorelation((0.374736336549233, 1.7827164615489603e-12), SpearmanrResult(correlation=0.37870118938702757, pvalue=9.953546864365974e-13), 6.232294617563739)

Model 2 coorelation((0.2809910972369881, 4.337209111172394e-07), SpearmanrResult(correlation=0.24450772731715217, pvalue=1.2139814391754527e-05), 11.3314447592068)

Model 3 coorelation((0.21728134260727436, 0.00012114960418890527), SpearmanrResult(correlation=0.17207124863438045, pvalue=0.002443981286046354), 12.747875354107649)


In [113]:
# Pick a random document from the test corpus and infer a vector from the model
import random

testcorps=dfQA['question']
doc_id = random.randint(0, len(testcorps) - 1)


inferred_vector = model1.infer_vector(testcorps[doc_id].split())

sims = model1.docvecs.most_similar([inferred_vector], topn=len(model1.docvecs))


print(doc_id)
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(testcorps[doc_id])))
print('suppose answer : "{}  "'.format(dfQA['context'][doc_id]))

print('For model 0 :')
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model1)
for label, index in [('MOST', 0)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(tg[int(sims[index][0])].words)))



4631
Test Document (4631): «W h a t   a l t i t u d e   d i d   t h e   S o l a r   R i s e r   r e a c h   i n   f e e t ?»

suppose answer : "In 1974, the unmanned AstroFlight Sunrise plane made the first solar flight. On 29 April 1979, the Solar Riser made the first flight in a solar-powered, fully controlled, man carrying flying machine, reaching an altitude of 40 feet (12 m). In 1980, the Gossamer Penguin made the first piloted flights powered solely by photovoltaics. This was quickly followed by the Solar Challenger which crossed the English Channel in July 1981. In 1990 Eric Scott Raymond in 21 hops flew from California to North Carolina using solar power. Developments then turned back to unmanned aerial vehicles (UAV) with the Pathfinder (1997) and subsequent designs, culminating in the Helios which set the altitude record for a non-rocket-propelled aircraft at 29,524 metres (96,864 ft) in 2001. The Zephyr, developed by BAE Systems, is the latest in a line of record-breaking so

In [126]:
from gensim.models.fasttext import FastText as FT

In [127]:
dt=df['process_context_WordTok']
ft_model = FT(dt,min_count=15,iter=50,sg=1)
ft_model.save("FT.model")
print("FastText Saved")


FastText Saved


In [128]:
print(ft_model.wv.similarity(w1='Beyonce', w2='album'))

0.545319


In [129]:
dfQA['Question_WordTok']=[word_tokenize(str(entry)) for entry in dfQA['question']]
new_word=dfQA['Question_WordTok']

In [130]:
modelft2 = FT.load("FT.model") #grab model
modelft2.build_vocab(new_word, update=True) #update vocab

In [131]:
from gensim.models.fasttext import FastText
nwModel=modelft2.train(new_word, total_examples=len(new_word),epochs=15)

In [132]:
modelft2.save("FT.modelQT")
print("FastText Saved")

FastText Saved


In [133]:
simQ=modelft2.wv.similarity(w1='Beyonce', w2='album')
simQ

0.5679518

In [134]:
dfQA['Answer_WordTok']=[word_tokenize(str(entry)) for entry in dfQA['answer']]
newWO=dfQA['Answer_WordTok']

In [135]:
modelft3 = FT.load("FT.modelQT") #grab model
modelft3.build_vocab(newWO, update=True) #update vocab

In [136]:
modelft3.train(newWO, total_examples=len(newWO), epochs=15)

In [138]:
simA=modelft3.wv.similarity(w1='Beyonce', w2='album')
simA

0.54094714

In [142]:
# n_similarity Compute cosine similarity between two sets of words
sc=[]
for i in range(len(dfQA['question'])):
        score = modelft3.n_similarity(dfQA['question'][i],dfQA['answer'][i])
        sc.append(score)
        
dfQA.insert(4, "simFT", sc, True)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  after removing the cwd from sys.path.


In [143]:
dfQA.head(15)

,context,question,answer,answerstart,simFT,is_impossible,Question_WordTok,Answer_WordTok
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s,269,0.845988,False,"[When, did, Beyonce, start, becoming, popular, ?]","[in, the, late, 1990s]"
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,207,0.911637,False,"[What, areas, did, Beyonce, compete, in, when,...","[singing, and, dancing]"
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,526,0.465760,False,"[When, did, Beyonce, leave, Destiny, 's, Child...",[2003]
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas",166,0.852045,False,"[In, what, city, and, state, did, Beyonce, gro...","[Houston, ,, Texas]"
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s,276,0.734666,False,"[In, which, decade, did, Beyonce, become, famo...","[late, 1990s]"
5,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what R&B group was she the lead singer?,Destiny's Child,320,0.910777,False,"[In, what, R, &, B, group, was, she, the, lead...","[Destiny, 's, Child]"
6,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What album made her a worldwide known artist?,Dangerously in Love,505,0.920547,False,"[What, album, made, her, a, worldwide, known, ...","[Dangerously, in, Love]"
7,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Who managed the Destiny's Child group?,Mathew Knowles,360,0.955528,False,"[Who, managed, the, Destiny, 's, Child, group, ?]","[Mathew, Knowles]"
8,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyoncé rise to fame?,late 1990s,276,0.737506,False,"[When, did, Beyoncé, rise, to, fame, ?]","[late, 1990s]"
9,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What role did Beyoncé have in Destiny's Child?,lead singer,290,0.909132,False,"[What, role, did, Beyoncé, have, in, Destiny, ...","[lead, singer]"
